In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Fork-Join Example 11
model = Network('model')

In [3]:
# Create network
delay = Delay(model, 'Delay1')
delay2 = Delay(model, 'Delay2')
queue1 = Queue(model,'Queue1',SchedStrategy.FCFS)
queue2 = Queue(model,'Queue2',SchedStrategy.FCFS)
fork = Fork(model,'Fork')
join = Join(model,'Join', fork)

jobclass1 = ClosedClass(model, 'class1', 10, delay, 0)

In [4]:
# Service configurations
queue1.set_service(jobclass1, Exp(1.0))
queue2.set_service(jobclass1, Exp(1.0))
delay.set_service(jobclass1, Exp(0.5))
delay2.set_service(jobclass1, Exp(2.0))

In [5]:
# Set routing matrix
P = model.init_routing_matrix()

# Serial routing through delays: delay -> delay2 -> fork
P.set(jobclass1, jobclass1, delay, delay2, 1.0)
P.set(jobclass1, jobclass1, delay2, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)

model.link(P)

In [6]:
# Solve with multiple methods
solvers = [SolverJMT(model, seed=23000), SolverMVA(model)]
for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.get_avg_table()

SOLVER 1: SolverJMT
JMT Model: /tmp/workspace/jsim/12359597666112809508/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 0.980834s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0  Delay1   class1  1.8026  1.8026  1.9628  1.9628  0.9450  0.9425
1  Delay2   class1  0.4646  0.4646  0.4845  0.4845  0.9425  0.9428
2  Queue1   class1  5.3478  0.9380  6.0091  3.0046  0.9428  0.9431
3  Queue2   class1  5.6791  0.9343  5.9441  2.9720  0.9428  0.9433
4    Join   class1  4.3491  0.0000  2.2717  2.2717  1.8720  0.9413
SOLVER 2: SolverMVA
MVA analysis [method: default(egflin), lang: java, env: 21.0.8] completed in 0.043077s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0  Delay1   class1  1.7184  1.7184  2.0000  2.0000  0.8592  0.8592
1  Delay2   class1  0.4296  0.4296  0.5000  0.5000  0.8592  0.8592
2  Queue1   class1  5.3091  0.8592  6.1792  3.0896  0.8592  0.8592
3  Queue2   class1  5.3091  0.8592  6.1792  3.0896  0.8592  0.8592
